<a href="https://colab.research.google.com/github/Birkbeck/msc-projects-2023-4-Gabriele_Monti_PEFT/blob/main/Llama_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction to LLaMA-Adapter Training on Colab
The LLaMA-Adapter project enables efficient fine-tuning of the LLaMA model to follow instructions using only 1.2M parameters. This guide will help you set up and train LLaMA-Adapter on Google Colab, similar to the Alpaca training process.
However, due to technical complexities and restricted access to LLaMA models, ensuring successful deployment and training on Colab requires careful preparation and adherence to detailed documentation.

https://github.com/OpenGVLab/LLaMA-Adapter?tab=readme-ov-file

https://github.com/OpenGVLab/LLaMA-Adapter

In [1]:
!pip install torch transformers


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [23]:
# Clone the GitHub repository "LLaMA-Adapter" from the specified URL.

!git clone https://github.com/OpenGVLab/LLaMA-Adapter.git

# Change the current working directory to "LLaMA-Adapter"
# and install the required Python packages listed in the "requirements.txt" file.
!cd LLaMA-Adapter && pip install -r requirements.txt



Cloning into 'LLaMA-Adapter'...
remote: Enumerating objects: 1151, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1151 (delta 10), reused 15 (delta 6), pack-reused 1126
Receiving objects: 100% (1151/1151), 34.40 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (543/543), done.


In [22]:
%cd /content
!rm -rf /content/LLaMA-Adapter

/content


In [3]:
# Use the wget command to download a file from the  URL.
# The -P option specifies the directory where the file should be saved.
# In this case, the file will be saved in the "LLaMA-Adapter/data" directory.

!wget -P LLaMA-Adapter/data https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json



--2024-07-26 13:28:16--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘LLaMA-Adapter/data/alpaca_data.json.1’

alpaca_data.json.1  100%[===================>]  21.72M  50.2MB/s    in 0.4s    

2024-07-26 13:28:17 (50.2 MB/s) - ‘LLaMA-Adapter/data/alpaca_data.json.1’ saved [22773992/22773992]



In [25]:
#it requires authentication for dowloading the weights


from huggingface_hub import notebook_login

notebook_login()



In [5]:
#get the model from HF

import os
from huggingface_hub import snapshot_download

# Specify the custom model path
custom_model_path = "/content/LLaMA-7B"
#nyanko7/LLaMA-7B

# Download the model to the specified directory without creating an additional subdirectory
model_path = snapshot_download(repo_id="nyanko7/LLaMA-7B", local_dir=custom_model_path, local_dir_use_symlinks=False)

print(f"The model is downloaded to: {model_path}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

checklist.chk:   0%|          | 0.00/100 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

consolidated.00.pth:   0%|          | 0.00/13.5G [00:00<?, ?B/s]

The model is downloaded to: /content/LLaMA-7B


In [26]:
#uses only 1000 examples to speed up the test

import json

# Path to your JSON file
file_path = '/content/LLaMA-Adapter/data/alpaca_data.json'

# Load the full dataset
with open(file_path, 'r') as file:
    data = json.load(file)

# Limit the number of examples
number_of_examples = 1000  # Set this to whatever number you need
limited_data = data[:number_of_examples]

# Save the limited dataset back to the same file
with open(file_path, 'w') as file:
    json.dump(limited_data, file)

print(f"Reduced dataset to {number_of_examples} examples and saved back to {file_path}")


Reduced dataset to 1000 examples and saved back to /content/LLaMA-Adapter/data/alpaca_data.json


In [36]:
#training loop

%cd /content/LLaMA-Adapter/alpaca_finetuning_v1

# Step 3: Setup paths (update with your actual paths)
TARGET_FOLDER = '/content/LLaMA-7B/'  # Adjusted to match the actual path
DATA_PATH = "/content/LLaMA-Adapter/data"  # Replace with your actual data path

# Prepare command
command = f"""
torchrun --nproc_per_node=1 --rdzv_endpoint=localhost:29501 finetuning.py \\
         --model Llama7B_adapter \\
         --llama_model_path {TARGET_FOLDER} \\
         --data_path {DATA_PATH}/alpaca_data.json \\
         --adapter_layer 10 \\
         --adapter_len 10 \\
         --max_seq_len 512 \\
         --batch_size 1 \\
         --epochs 1 \\
         --warmup_epochs 1 \\
         --blr 9e-3 \\
         --weight_decay 0.02 \\
         --output_dir ./checkpoint/
"""

# Execute command
!{command}


/content/LLaMA-Adapter/alpaca_finetuning_v1
2024-07-25 19:19:28.787961: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 19:19:28.788010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 19:19:28.789316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 19:19:28.796023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-2

The setup for running LLaMA with an adapter is complex, plagued by poor documentation and technical errors, such as a mismatch in function parameters causing crashes. Access to LLaMA models is restricted by Meta, adding further challenges for practitioners without high-end hardware, making Colab a crucial resource.

In [55]:
#inference loop that needs investigation

%cd /content/LLaMA-Adapter/alpaca_finetuning_v1

# Define the paths
TARGET_FOLDER = '/content/LLaMA-7B/'
ADAPTER_PATH = '/content/LLaMA-Adapter/alpaca_finetuning_v1/checkpoint/checkpoint-0.pth'  # Make sure to define your adapter path

# Prepare the command
command = f"""
torchrun --nproc_per_node=1 --rdzv_endpoint=localhost:29501 /content/LLaMA-Adapter/example.py \
          --ckpt_dir {TARGET_FOLDER}/ \
          --tokenizer_path {TARGET_FOLDER}/tokenizer.model \
          --adapter_path {ADAPTER_PATH}

"""
!{command}









/content/LLaMA-Adapter/alpaca_finetuning_v1
> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loading
[rank0]: Traceback (most recent call last):
[rank0]:   File "/content/LLaMA-Adapter/example.py", line 117, in <module>
[rank0]:     fire.Fire(main)
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 143, in Fire
[rank0]:     component_trace = _Fire(component, args, parsed_flag_args, context, name)
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 477, in _Fire
[rank0]:     component, remaining_args = _CallAndUpdateTrace(
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/fire/core.py", line 693, in _CallAndUpdateTrace
[rank0]:     component = fn(*varargs, **kwargs)
[rank0]:   File "/content/LLaMA-Adapter/example.py", line 95, in main
[rank0]:     generator = load(ckpt_dir, tokenizer_path, adapter_path, local_rank, world_size, max_seq_len, max_batch_size, quantizer)
[ran